In [1]:
import json
import config
from os import listdir

In [3]:
def Transforme(s):
    repetiveis = config.MARC_REPETIVEL

    controfield = {}
    for i in s['controfield']:
        controfield[i['code']] = i['value']

    
    datafield =  {}
    for i in s['datafield']:
        subfield = {}
        for k, v in i['subfield'].items():
            subfield[k] = v.rstrip()
        if i['code'] not in repetiveis:
            datafield[i['code']] = subfield
        else:
            if i['code'] not in datafield.keys():
                datafield[i['code']] = [subfield]
            else:
                datafield[i['code']].append(subfield)
    
    marcJson = {'controfield': controfield, 'datafield': datafield}
    return marcJson

In [ ]:
mes = 'mar'
registros = listdir(r"out\mcti\2005")
for i in registros:
    path = f'out\mcti\{mes}/{i}'
    with open(path, encoding="utf-8") as file_json:
        s = json.loads(file_json.read())
    marc = Transforme(s)
    with open(
        f"out\mcti\marc\2005/{i}","w", encoding='utf-8') as jsonfile:
        json.dump(marc, jsonfile, ensure_ascii=False, indent=4)
    print(i)

In [6]:
registros = listdir(r"out\mcti\2005")

In [ ]:
path = r"out\mcti\2006"
registros = listdir(path)
for registro in registros:
    pathfile = f'out\mcti/2006\{registro}'
    with open(pathfile, encoding="utf-8") as file_json:
        s = json.loads(file_json.read())
        file_json.close()
    marc = Transforme(s)
    with open(
        f"out\mcti\marc/2006/{registro}","w", encoding='utf-8') as jsonfile:
        json.dump(marc, jsonfile, ensure_ascii=False, indent=4)
        jsonfile.close()
    print(registro)

In [31]:
def ParserMarc700(v, solr_doc):
    for i in v:
        tipo = i["e"]
        print(tipo)
        responsabilty = f'marc700{tipo}'
        if responsabilty not in solr_doc.keys():
            solr_doc[responsabilty] = [tipo]
        else:
            solr_doc[responsabilty].append(tipo)
    return solr_doc
    

In [13]:
def ParserMarc6XX(k, v, solr_doc):
    for i in v:
        
        #Parser Simple
        simple = f'marc{k}a'
        if simple not in solr_doc.keys():
            solr_doc[simple] = [i.get('a')]
        else:
            solr_doc[simple].append(i.get('a'))
        
        #Parser Full
        subfields = list()
        for _, vs in i.items():
            subfields.append(vs)
        subfields = "--".join(subfields)
        full = f'marc{k}completo'
        if full not in solr_doc.keys():
            solr_doc[full] = [subfields]
        else:
            solr_doc[full].append(subfields)
    return solr_doc

In [28]:
def ParserSolr(s, institution):

    solr_doc = {
        'id': int(s.get('controfield').get('001')),
        'institution': institution
    }

    for k, v in s.get('datafield').items():
        if isinstance(v, dict):
            for ks, vs in v.items():
                solr_doc[f'marc{k}{ks}'] = vs
        else:
            #ParserMarc700
            if k == '700':
                solr_doc = ParserMarc700(v, solr_doc)
            #ParserMarc6XX
            elif k[0] == '6':
                solr_doc = ParserMarc6XX(k, v, solr_doc)
            else:
                for subfield in v:
                    for ks, vs in subfield.items():
                        tag = f'marc{k}{ks}'
                        if tag not in solr_doc.keys():
                            solr_doc[tag] = [vs]
                        else:
                            solr_doc[tag].append(vs)
        
    return solr_doc
#solr_doc = ParserSolr(s, 'Goeldi')

In [2]:
with open(r"C:\Users\inaci\Desktop\Une\une\une-crawler\out\mcti\codesNov2008.json", encoding="utf-8") as file_json:
    codes = json.loads(file_json.read())
    file_json.close()

In [17]:
solr_doc = ParserSolr(s, 'Mcti')

In [30]:
docs = list()
path = r"out\mcti\2005"
registros = listdir(r'out\mcti\marc\2005')
for registro in registros:
    pathfile = f'out\mcti\marc/2005\{registro}'
    with open(pathfile, encoding="utf-8") as file_json:
        s = json.loads(file_json.read())
        solr_doc = ParserSolr(s, 'Mcti')
        docs.append(solr_doc)
        file_json.close()
    #print(registro)

Coautor
Coautor
Coautor
Coautor
Editor
Editor
Editor
Coordenador
Supervisor
Supervisor
Supervisor
Editor
Editor
Coautor
Coautor
Editor
Editor
Coautor
Coautor
Coautor
Coordenador
Coautor
Organizador
Coautor
Organizador
Organizador
Coautor
Coautor
Coautor
Coautor
Organizador
Editor
Editor
Editor
Editor
Organizador
Organizador
Organizador
Coautor
Coautor
Coautor
Coautor
Coautor
Coautor
Coautor
Coautor
Coautor
Coautor
Coautor
Coautor
Coautor
Coautor
Coautor
Coautor
Coautor
Coautor
Coautor
Coautor
Organizador
Organizador
Organizador
Coautor
Coautor
Coautor
Coautor
Coautor
Organizador
Organizador
Organizador
Organizador
Coautor
Coautor
Coautor
Coautor
Coautor
Coautor
Organizador
Organizador
Coautor
Coordenador
Coordenador
Coordenador
Coautor
Coautor
Coautor
Coautor
Coautor
Coautor
Coautor
Coautor
Coautor
Coautor
Coautor
Coautor
Coautor
Coordenador
Coordenador
Coordenador
Coautor
Coautor
Coautor
Coautor
Coautor
Coautor
Coautor
Coautor
Coautor
Coautor
Coautor
Organizador
Organizador
Organizado

In [22]:
len(docs)

1049

In [26]:
with open(
        r"out/mcti/docs/doc_2005.json","w", encoding='utf-8') as jsonfile:
        json.dump(docs, jsonfile, ensure_ascii=False, indent=4)
        jsonfile.close()

CREATE SOLR DOC

In [ ]:
path = r'out\goeldi\01-09-22_30-09-22'
listMarcJson = listdir(path)

docs = list()
for i in listMarcJson:
    with open(f'{path}/{i}', encoding="utf-8") as file_json:
        s = json.loads(file_json.read())
        solr_doc = ParserSolr(s, 'Goeldi')
        docs.append(solr_doc)
        #print(i)
        file_json.close()

with open(
    f"out/goeldi/solr.json", "w", encoding='utf-8') as file_json:
    json.dump(docs, file_json, ensure_ascii=False, indent=4)
    file_json.close()

In [9]:
codes['codes'][42:]

['23543',
 '9222',
 '9279',
 '9204',
 '9252',
 '9172',
 '9282',
 '9287',
 '9166',
 '9165',
 '9267',
 '9194',
 '9248',
 '9203',
 '9161',
 '9162',
 '9286',
 '9187',
 '9188',
 '9216',
 '9280',
 '9215',
 '9249',
 '9220',
 '9251',
 '9212',
 '9276',
 '9246',
 '9195',
 '9262',
 '9270',
 '9184',
 '9217',
 '9199',
 '9183',
 '9196',
 '9168',
 '9169',
 '9223',
 '9292',
 '9160',
 '9171',
 '9200',
 '9198',
 '9201',
 '18282',
 '18283',
 '18284',
 '18288',
 '18270',
 '18272',
 '18285',
 '18278',
 '18279',
 '18280',
 '18281',
 '18277',
 '9258',
 '9274',
 '9190',
 '9225',
 '9177',
 '9264',
 '9221',
 '9197',
 '9245',
 '9182',
 '9266',
 '9260',
 '9257',
 '9179',
 '9170',
 '9233',
 '9268',
 '9265',
 '9281',
 '9175',
 '9277',
 '9214',
 '9271',
 '9224',
 '9240',
 '9243',
 '9241',
 '9244',
 '9263',
 '9229',
 '9290',
 '9232',
 '9230',
 '9237']